### Урок 2. Регрессия на практике
Ссылка на набор данных.

Пусть у нас имеется некоторый набор данных sales.csv о продажах квартир.
— цена продажи (SalePrice, зависимая переменная),
— наземная жилая зона (GrLivArea),
— общее качество (Overall Quality),
— территория гаража (GarageArea),
— условия сделки (SaleCondition).
Реализуйте построение модели линейной регрессии на этих данных. Проверьте качество модели на обучающей и тестовой выборках с помощью MAE, MSE.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

Загрузка данных

In [2]:
data = pd.read_csv('sales.csv')
data.head()

,SalePrice,GrLivArea,GarageArea,OverallQual,Street,SaleCondition
0,208500,1710,548,7,Pave,Normal
1,181500,1262,460,6,Pave,Normal
2,223500,1786,608,7,Pave,Normal
3,140000,1717,642,7,Pave,Abnorml
4,250000,2198,836,8,Pave,Normal


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SalePrice      1460 non-null   int64 
 1   GrLivArea      1460 non-null   int64 
 2   GarageArea     1460 non-null   int64 
 3   OverallQual    1460 non-null   int64 
 4   Street         1460 non-null   object
 5   SaleCondition  1460 non-null   object
dtypes: int64(4), object(2)
memory usage: 68.6+ KB


In [4]:
data.duplicated().sum()

1

In [5]:
data = data.drop_duplicates()

In [6]:
data.duplicated().sum()

0

In [8]:
#Посмотрим количественные признаки
data.describe()

,SalePrice,GrLivArea,GarageArea,OverallQual
count,1459.000000,1459.000000,1459.000000,1459.000000
mean,180941.703907,1515.679918,472.975326,6.098698
std,79465.875738,525.595586,213.878071,1.383270
min,34900.000000,334.000000,0.000000,1.000000
25%,129950.000000,1129.000000,333.000000,5.000000
50%,163000.000000,1464.000000,480.000000,6.000000
75%,214000.000000,1777.500000,576.000000,7.000000
max,755000.000000,5642.000000,1418.000000,10.000000


In [10]:
#Посмотрим каиегориальные признаки
data.describe(include='object')

,Street,SaleCondition
count,1459,1459
unique,2,6
top,Pave,Normal
freq,1453,1197


In [11]:
#Закодируем категориальные признаки. 
data['Street'].value_counts()

Street
Pave    1453
Grvl       6
Name: count, dtype: int64

In [18]:
#Мы имеем всего 2 улицы, пусть реже всречаемая будет 0
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['Street_encoded'] = label_encoder.fit_transform(data.Street)
data.head()

,SalePrice,GrLivArea,GarageArea,OverallQual,Street,SaleCondition,Street_encoded
0,208500,1710,548,7,Pave,Normal,1
1,181500,1262,460,6,Pave,Normal,1
2,223500,1786,608,7,Pave,Normal,1
3,140000,1717,642,7,Pave,Abnorml,1
4,250000,2198,836,8,Pave,Normal,1


In [19]:
sale_condition_counts = data['SaleCondition'].value_counts()
sale_condition_counts

SaleCondition
Normal     1197
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: count, dtype: int64

In [20]:
#признак SaleCondition имеет 6 категорий, закодируем его  по частоте свтречаемости
data['SaleCondition_encoded'] = data['SaleCondition'].map(sale_condition_counts)
data.head()

,SalePrice,GrLivArea,GarageArea,OverallQual,Street,SaleCondition,Street_encoded,SaleCondition_encoded
0,208500,1710,548,7,Pave,Normal,1,1197
1,181500,1262,460,6,Pave,Normal,1,1197
2,223500,1786,608,7,Pave,Normal,1,1197
3,140000,1717,642,7,Pave,Abnorml,1,101
4,250000,2198,836,8,Pave,Normal,1,1197


In [21]:
data.drop(['Street', 'SaleCondition'], axis=1, inplace=True)

In [22]:
data.head()

,SalePrice,GrLivArea,GarageArea,OverallQual,Street_encoded,SaleCondition_encoded
0,208500,1710,548,7,1,1197
1,181500,1262,460,6,1,1197
2,223500,1786,608,7,1,1197
3,140000,1717,642,7,1,101
4,250000,2198,836,8,1,1197


In [66]:
#сделаем стандартизацию данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
columns_for_scaling = ['SalePrice',	'GrLivArea',	'GarageArea',	'OverallQual']
scaled_data = pd.concat([pd.DataFrame(scaler.fit_transform(data[columns_for_scaling]), columns=columns_for_scaling), data[['Street_encoded', 'SaleCondition_encoded']]], axis=1)
scaled_data.head()

,SalePrice,GrLivArea,GarageArea,OverallQual,Street_encoded,SaleCondition_encoded
0,0.346913,0.369841,0.350903,0.651797,1.0,1197.0
1,0.007028,-0.482818,-0.060688,-0.071376,1.0,1197.0
2,0.535738,0.514488,0.631533,0.651797,1.0,1197.0
3,-0.515388,0.383164,0.790556,0.651797,1.0,101.0
4,0.869329,1.298630,1.697926,1.374970,1.0,1197.0


In [67]:
scaled_data.Street_encoded.value_counts()

Street_encoded
1.0    1453
0.0       6
Name: count, dtype: int64

In [68]:
#Зададим пространство признаков и целевую переменную
X = data.drop(['SalePrice'], axis=1)
y = data['SalePrice']

In [69]:
#Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
#Создание модели линейной регрессии
model = LinearRegression()

In [71]:
#Обучение модели
model.fit(X_train, y_train)

LinearRegression()

In [72]:
#Предсказание на тестовой выборке
y_pred = model.predict(X_test)

In [73]:
#Вычисление MAE и MSE на обучающей выборке
y_train_pred = model.predict(X_train)
mae_train = mean_absolute_error(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)

In [74]:
#Вычисление MAE и MSE на тестовой выборке
mae_test = mean_absolute_error(y_test, y_pred)
mse_test = mean_squared_error(y_test, y_pred)

In [75]:
print(f"MAE на обучающей выборке: {mae_train:.2f}")
print(f"MSE на обучающей выборке: {mse_train:.2f}")
print(f"MAE на тестовой выборке: {mae_test:.2f}")
print(f"MSE на тестовой выборке: {mse_test:.2f}")

MAE на обучающей выборке: 27264.23
MSE на обучающей выборке: 1761242499.48
MAE на тестовой выборке: 25737.70
MSE на тестовой выборке: 1156784458.10


In [76]:
data.SalePrice.mean()

180941.70390678546

MAE (средняя абсолютная ошибка) на тестовой выборке 25737.70, чем MAE на обучающей выборке 27264.23, одинаковый порядок величин говорит от обсутствии переобучения.  Модель показывает значительное улучшение на тестовой выборке по сравнению с обучающей. При средней стоимости квартир по всему датасету 181000 такая mae составляет 13%, что приемлемо для такой простой модели. Для более точного прогноза стоит сравнить результаты с другими алгоритмами машинного обучения. 